In [1]:
import os
import pandas as pd
import numpy as np
import time
import json
import ast
import re
import google.generativeai as palm
from dotenv import load_dotenv

import sys
sys.path.insert(0, '../serpapi_data_ingestion')
sys.path.insert(1, '../snowflake')
from snowflake_data import *
###############################################################################
# Python Functions - Custom
# import processing_fxns
# import similarity_grouping_fxn


###############################################################################
# Load variables from .env file
load_dotenv('C:\\Users\\j.videlefsky\\Documents\\DAMG7374 - GenAI and DataEng\\BiteBuddy\\.env')
# Access variables
PALM_API_KEY = os.getenv("PALM_API_KEY")

###############################################################################
# LLM - Configure the PaLM API with your API key.
palm.configure(api_key=PALM_API_KEY)

c:\Users\j.videlefsky\Anaconda3\envs\damg7374\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-12-13 17:15:40.963 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager


In [3]:
###############################################################################
# Import Data from Snowflake
business_name = "Oliveira's Steak House"
business_name = business_name.replace("'", "''")
# test = f"""SELECT business_name, review_text
#                         FROM DAMG7374.staging.sample_reviews
#                         WHERE BUSINESS_NAME = '{business_name}'
#                         LIMIT 10"""
df = get_reviews_new(business_name)

###############################################################################
# Data Exploration
# Display the DataFrame
df.head()

selected_restaurant - get_reviews_summary: Oliveira''s Steak House


,BUSINESS_NAME,RATING,REVIEW_TEXT
0,Oliveira's Steak House,3,"Meat is a little salty for my taste, but wasn'..."
1,Oliveira's Steak House,5,Delicious and authentic Brazilian food. Picanh...
2,Oliveira's Steak House,4,The all you can eat buffet was amazing. Sirloi...
3,Oliveira's Steak House,5,My favorite place in the Boston area. Best ro...
4,Oliveira's Steak House,5,"Buffet offers a big, varied selection to compl..."


In [4]:
df.shape
# Estimated code run time - 90 seconds to process 90 rows
print(f"Estimated run time: {round((df.shape[0] / 90 * 90) / 60,2)} minutes")

Estimated run time: 20.78 minutes


In [5]:
# testing
df = df.head(100)
df.index

RangeIndex(start=0, stop=100, step=1)

In [6]:
###############################################################################
# LLM Function - Get Meal Names and Sentiment
def get_meal_names(review_text):
    """Generates a list of meal names from a review text using the PaLM API.

    Args:
        review_text: A string containing the review text.

    Returns:
        A list of strings containing the meal names.
    """

    # Select a PaLM 2 model.
    models = [m for m in palm.list_models() if 'generateText' in m.supported_generation_methods]
    model = models[0].name # models/text-bison-001

    # Set temperature (randomness of model)
    temperature = 0

    # Generate text using the PaLM 2 model.
    prompt = f"""
    Your task is to perform the following actions:
    1 - Extract each meal names and it's associated sentiment from the text delimited by triple backticks below. 
    2 - Use a sentiment scale from 0 to 1, where 0 is the most negative sentiment and 1 is the most positive sentiment.
    3 - Output as a list of lists in format [["meal names", sentiment]]
    4 - If the text does not contain a meal name, then output exactly "No Meals in Review".

    Text:
    ```{review_text}```
    """

    response = palm.generate_text(prompt=prompt, model=model, temperature=temperature)
    print(f'LLM Response: {response.result}')

    # Incorrect response format from LLM
    try:
        # Extract the meal names and sentiment from the generated text
        meal_names = ast.literal_eval(response.result)
    except:
        meal_names = ['PaLM Response is in incorrect format']
        
    # No meals in review
    if "No Meals in Review" in str(response.result):
        meal_names = ['No Meals in Review']

    return meal_names

In [7]:
###############################################################################
# CODE FLOW:
review_column = 'REVIEW_TEXT'
llm_output_column = 'MEALS_AND_SENTIMENTS'

def process_reviews(df, review_column='REVIEW_TEXT', llm_output_column='MEALS_AND_SENTIMENTS'):
    # 1. Processing Chunks - to apply the llm to the review_text column 90 records per minute
    # Chunk size limited by LLM API
    chunk_size = 90

    def process_chunk(chunk):
        # Create a new column to store the meal names.
        chunk['MEALS_AND_SENTIMENTS'] = chunk[review_column].apply(get_meal_names)
        return chunk

    # Record the start time
    start_time = time.time()

    # Apply the function in chunks with a 60 second break
    result_df = pd.DataFrame()

    for _, chunk in df.groupby(df.index // chunk_size):
        print(f'Chunk: {_}, Chunk_Size: {chunk.index}')
        processed_chunk = process_chunk(chunk)
        result_df = pd.concat([result_df, processed_chunk])
        print(f"""\n 65 Second break between processing records, due to 90 requests/minute quota. \n
            Current Records Processed: {result_df.shape[0]} \n""")
        # if more than 90 records, then 65 second break
        if processed_chunk.shape[0] >= 90:
            time.sleep(65)  # 2 minutes break

    # Record the stop time
    stop_time = time.time()

    # Calculate the elapsed time
    elapsed_time = stop_time - start_time

    print("Elapsed time:", elapsed_time, "seconds")

    return result_df

result_df = process_reviews(df, review_column, llm_output_column)

Chunk: 0, Chunk_Size: Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
       54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71,
       72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89],
      dtype='int64')
LLM Response: [['Garlic beef', 0.8], ['Flank steak', 0.8], ['Rice and beans', 1.0]]
LLM Response: [['Picanha', 1], ['Desserts', 1]]
LLM Response: [['all you can eat buffet', 1], ['Sirloin', 1]]
LLM Response: [['rodizio', 1]]
LLM Response: [['Buffet', 1], ['Pork ribs', 1]]
LLM Response: No Meals in Review
LLM Response: [['cheese steak', 1], ['cinnamon pineapple', 1], ['all you can eat option', 1]]
LLM Response: No Meals in Review
LLM Response: [['buffet', 1], ['meat skewers', 1]]
LLM Response: [['No Meals in Review']]
LLM Response: No Meals in Review
LLM R

In [8]:
# drop columns
# result_df = result_df.drop(columns=['MEAL_NAME', 'SENTIMENT'])
result_df.head()
# result_df.shape

,BUSINESS_NAME,RATING,REVIEW_TEXT,MEALS_AND_SENTIMENTS
0,Oliveira's Steak House,3,"Meat is a little salty for my taste, but wasn'...","[[Garlic beef, 0.8], [Flank steak, 0.8], [Rice..."
1,Oliveira's Steak House,5,Delicious and authentic Brazilian food. Picanh...,"[[Picanha, 1], [Desserts, 1]]"
2,Oliveira's Steak House,4,The all you can eat buffet was amazing. Sirloi...,"[[all you can eat buffet, 1], [Sirloin, 1]]"
3,Oliveira's Steak House,5,My favorite place in the Boston area. Best ro...,"[[rodizio, 1]]"
4,Oliveira's Steak House,5,"Buffet offers a big, varied selection to compl...","[[Buffet, 1], [Pork ribs, 1]]"


In [52]:
df = result_df.copy()
# Use apply along with pd.Series to split the lists into two columns
df[['MEAL_NAME', 'SENTIMENT']] = df[llm_output_column].apply(lambda x: pd.Series([x[0], x[1]] if len(x) == 2 else ['', .5]))
df.head()

,BUSINESS_NAME,RATING,REVIEW_TEXT,MEALS_AND_SENTIMENTS,MEAL_NAME,SENTIMENT
0,Oliveira's Steak House,3,"Meat is a little salty for my taste, but wasn'...","[Garlic beef, 0.8]",Garlic beef,0.8
0,Oliveira's Steak House,3,"Meat is a little salty for my taste, but wasn'...","[Flank steak, 0.8]",Flank steak,0.8
0,Oliveira's Steak House,3,"Meat is a little salty for my taste, but wasn'...","[Rice and beans, 1.0]",Rice and beans,1.0
1,Oliveira's Steak House,5,Delicious and authentic Brazilian food. Picanh...,"[Picanha, 1]",Picanha,1.0
1,Oliveira's Steak House,5,Delicious and authentic Brazilian food. Picanh...,"[Desserts, 1]",Desserts,1.0


In [53]:
###############################################################################
# POST-PROCESSING:
def post_processing(df, llm_output_column='MEALS_AND_SENTIMENTS'):
    # Use explode to transform lists into separate rows
    df = df.explode(llm_output_column)

    # Use apply along with pd.Series to split the lists into two columns
    df[['MEAL_NAME', 'SENTIMENT']] = df[llm_output_column].apply(lambda x: pd.Series([x[0], x[1]] if len(x) == 2 else ['', .5]))
    
    # When the LLM messes up the format
    # Convert lists to strings and remove square brackets
    df[['MEAL_NAME', 'SENTIMENT']] = df[['MEAL_NAME', 'SENTIMENT']].apply(lambda x: str(x).strip('[]') if type(x) is list else x)

    # Apply the function to the text column 
    # df['MEAL_NAME'] = df['MEAL_NAME'].apply(processing_fxns.remove_stop_words)

    return df

result_df = post_processing(df, llm_output_column)

TypeError: object of type 'float' has no len()

In [56]:
result_df = df
result_df.head()

,BUSINESS_NAME,RATING,REVIEW_TEXT,MEALS_AND_SENTIMENTS,MEAL_NAME,SENTIMENT
0,Oliveira's Steak House,3,"Meat is a little salty for my taste, but wasn'...","[Garlic beef, 0.8]",Garlic beef,0.8
0,Oliveira's Steak House,3,"Meat is a little salty for my taste, but wasn'...","[Flank steak, 0.8]",Flank steak,0.8
0,Oliveira's Steak House,3,"Meat is a little salty for my taste, but wasn'...","[Rice and beans, 1.0]",Rice and beans,1.0
1,Oliveira's Steak House,5,Delicious and authentic Brazilian food. Picanh...,"[Picanha, 1]",Picanha,1.0
1,Oliveira's Steak House,5,Delicious and authentic Brazilian food. Picanh...,"[Desserts, 1]",Desserts,1.0


In [57]:
#################################################################################
# FUNCTION FOR CLUSTERING
# pip install sentence-transformers scikit-learn pandas
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering, KMeans
import pandas as pd


# Load a pre-trained BERT model from Sentence Transformers
model = SentenceTransformer('all-MiniLM-L6-v2') #('gte-tiny') #('paraphrase-MiniLM-L6-v2')

def clustering(df, column_name=llm_output_column, model=model, cluster_percentage=50):
    
    # Get embeddings for the terms
    embeddings = model.encode(df[column_name].astype(str).tolist(), convert_to_tensor=True)
    
    # Perform clustering using Agglomerative Clustering
    # Adjust the number of clusters (n_clusters) based on your use case
    unique_values = len(df['MEAL_NAME'].unique())
    # 66.7% of the unique values
    n_clusters = int(unique_values * cluster_percentage/100)
    # clustering = AgglomerativeClustering(n_clusters=n_clusters, affinity='cosine', linkage='average')
    clustering = KMeans(n_clusters=n_clusters)
    df['CLUSTER'] = clustering.fit_predict(embeddings)
    
    # Display the grouped DataFrame
    print(df)

    return df
    
df = clustering(result_df, llm_output_column, model)

c:\Users\j.videlefsky\Anaconda3\envs\damg7374\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\j.videlefsky\Anaconda3\envs\damg7374\Lib\site-packages\sklearn\cluster\_kmeans.py:1440: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


             BUSINESS_NAME  RATING  \
0   Oliveira's Steak House       3   
0   Oliveira's Steak House       3   
0   Oliveira's Steak House       3   
1   Oliveira's Steak House       5   
1   Oliveira's Steak House       5   
..                     ...     ...   
95  Oliveira's Steak House       5   
96  Oliveira's Steak House       5   
97  Oliveira's Steak House       5   
98  Oliveira's Steak House       4   
99  Oliveira's Steak House       5   

                                          REVIEW_TEXT   MEALS_AND_SENTIMENTS  \
0   Meat is a little salty for my taste, but wasn'...     [Garlic beef, 0.8]   
0   Meat is a little salty for my taste, but wasn'...     [Flank steak, 0.8]   
0   Meat is a little salty for my taste, but wasn'...  [Rice and beans, 1.0]   
1   Delicious and authentic Brazilian food. Picanh...           [Picanha, 1]   
1   Delicious and authentic Brazilian food. Picanh...          [Desserts, 1]   
..                                                ...          

In [59]:
df.head()
# print total records
print(f'Total Records: {df.shape[0]}')
# print unique records of MEAL_NAME
print(f'Unique Records: {df["MEAL_NAME"].nunique()}')
# print unique records of CLUSTER
print(f'Unique Records: {df["CLUSTER"].nunique()}')

Total Records: 126
Unique Records: 52
Unique Records: 26


In [65]:
#################################################################################
# Assign Labels to Clusters
def assign_cluster_labels(df):
    # Group by 'BUSINESS_NAME' and 'CLUSTER'
    grouped_df = df.groupby(['BUSINESS_NAME', 'CLUSTER'])

    cluster_labels = grouped_df['MEAL_NAME'].apply(lambda x: x.mode().values[0]).reset_index()

    # Rename the column for clarity
    cluster_labels = cluster_labels.rename(columns={'MEAL_NAME': 'CLUSTER_LABEL'})

    # Merge the result back to the original DataFrame
    final_df = pd.merge(df, cluster_labels, on=['BUSINESS_NAME', 'CLUSTER'], how='left')
    
    # final_df = final_df.drop(columns=['CLUSTER_LABEL'])

    return final_df

df = assign_cluster_labels(df)

In [66]:
df.head()

,BUSINESS_NAME,RATING,REVIEW_TEXT,MEALS_AND_SENTIMENTS,MEAL_NAME,SENTIMENT,CLUSTER,CLUSTER_LABEL
0,Oliveira's Steak House,3,"Meat is a little salty for my taste, but wasn'...","[Garlic beef, 0.8]",Garlic beef,0.8,3,Flank steak
1,Oliveira's Steak House,3,"Meat is a little salty for my taste, but wasn'...","[Flank steak, 0.8]",Flank steak,0.8,3,Flank steak
2,Oliveira's Steak House,3,"Meat is a little salty for my taste, but wasn'...","[Rice and beans, 1.0]",Rice and beans,1.0,18,food
3,Oliveira's Steak House,5,Delicious and authentic Brazilian food. Picanh...,"[Picanha, 1]",Picanha,1.0,13,Picanha
4,Oliveira's Steak House,5,Delicious and authentic Brazilian food. Picanh...,"[Desserts, 1]",Desserts,1.0,14,Desserts


In [5]:
# df2 = df[['BUSINESS_NAME', 'RATING', 'MEAL_NAME', 'SENTIMENT', 'CLUSTER', 'CLUSTER_LABEL']]
# df2.head()

# write df to csv
# df.to_csv('test.csv', index=False)

# df where MEAL_NAME is NAN
df = df[~df['MEAL_NAME'].isnull()]

df.head()


,BUSINESS_NAME,RATING,REVIEW_TEXT,MEALS_AND_SENTIMENTS,MEAL_NAME,SENTIMENT,CLUSTER,CLUSTER_LABEL
0,Oliveira's Steak House,3,"Meat is a little salty for my taste, but wasn'...","['Garlic beef', 0.8]",Garlic beef,0.8,3,Flank steak
1,Oliveira's Steak House,3,"Meat is a little salty for my taste, but wasn'...","['Flank steak', 0.8]",Flank steak,0.8,3,Flank steak
2,Oliveira's Steak House,3,"Meat is a little salty for my taste, but wasn'...","['Rice and beans', 1.0]",Rice and beans,1.0,18,food
3,Oliveira's Steak House,5,Delicious and authentic Brazilian food. Picanh...,"['Picanha', 1]",Picanha,1.0,13,Picanha
4,Oliveira's Steak House,5,Delicious and authentic Brazilian food. Picanh...,"['Desserts', 1]",Desserts,1.0,14,Desserts


In [6]:
#################################################################################
# Upload the data to Snowflake
# append snowflake table damg7374.mart.reviews_llm_output with data in df
# df.to_sql('reviews_llm_output', engine, schema='damg7374.mart', if_exists='append', index=False)
# read data from csv file into pandas dataframe
# df = pd.read_csv('test.csv')
df.head()
update_reviews(df, 'damg7374.mart.review_llm_output')

,BUSINESS_NAME,RATING,MEAL_NAME,SENTIMENT,CLUSTER,CLUSTER_LABEL
0,Oliveira's Steak House,3,Garlic beef,0.8,3,Flank steak
1,Oliveira's Steak House,3,Flank steak,0.8,3,Flank steak
2,Oliveira's Steak House,3,Rice and beans,1.0,18,food
3,Oliveira's Steak House,5,Picanha,1.0,13,Picanha
4,Oliveira's Steak House,5,Desserts,1.0,14,Desserts
...,...,...,...,...,...,...
92,Oliveira's Steak House,4,Good food.,1.0,2,Great food
98,Oliveira's Steak House,5,food,1.0,5,food
99,Oliveira's Steak House,4,food,0.8,18,food
100,Oliveira's Steak House,4,drinks,0.2,9,drinks
